In this exercise, we will look at how to slide and dice data using Pandas.

In [2]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/movielens/movies.dat', sep='::', names=['index', "movie", "genre"], index_col=0,engine='python')
df.head()

,movie,genre
index,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


## iloc : Position based accessor

We will look at Pandas accessor iloc which uses integer indexes 

In [3]:
# We have set index as a numeric value, so we can access any specific row as follows
print("1st record in our list is:\n")
df.iloc[0]

1st record in our list is:



movie               Toy Story (1995)
genre    Animation|Children's|Comedy
Name: 1, dtype: object

In [4]:
#If we want to access the genre of a specific movie we also mention the column index 
df.iloc[3,1]

'Comedy|Drama'

In [5]:
# We can split the genre as a list and assign it back to the dataframe 
df.genre = df.genre.str.split("|")
df.head()

,movie,genre
index,,
1,Toy Story (1995),"[Animation, Children's, Comedy]"
2,Jumanji (1995),"[Adventure, Children's, Fantasy]"
3,Grumpier Old Men (1995),"[Comedy, Romance]"
4,Waiting to Exhale (1995),"[Comedy, Drama]"
5,Father of the Bride Part II (1995),[Comedy]


In [6]:
#Suppose we know we need genre information for movies 2 to 5. We can do this as follows
df.iloc[1:5,1]

index
2    [Adventure, Children's, Fantasy]
3                   [Comedy, Romance]
4                     [Comedy, Drama]
5                            [Comedy]
Name: genre, dtype: object

## loc : Label based accessor
Now we will look at label based accessor loc

In this exercise, we will load 2017 EPL data for one of the club, and then create a clean dataset for further analysis

In [3]:
# We will load the 2017 English Premier League data (http://www.football-data.co.uk/)
epl = pd.read_csv('./data/EPL/2017.csv')
epl.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,Unnamed: 65,Unnamed: 66,Unnamed: 67
0,E0,11/08/17,Arsenal,Leicester,4,3,H,2,2,D,...,1.91,1.85,2.10,2.02,1.49,4.73,7.25,NaN,NaN,NaN
1,E0,12/08/17,Brighton,Man City,0,2,A,0,0,D,...,1.95,1.91,2.01,1.96,11.75,6.15,1.29,NaN,NaN,NaN
2,E0,12/08/17,Chelsea,Burnley,2,3,A,0,3,A,...,2.03,1.97,1.95,1.90,1.33,5.40,12.25,NaN,NaN,NaN
3,E0,12/08/17,Crystal Palace,Huddersfield,0,3,A,0,2,A,...,2.10,2.05,1.86,1.83,1.79,3.56,5.51,NaN,NaN,NaN
4,E0,12/08/17,Everton,Stoke,1,0,H,1,0,H,...,1.94,1.90,2.01,1.98,1.82,3.49,5.42,NaN,NaN,NaN


In [4]:
# Let's select only a few columns we are interested in
df2 = epl.loc[: , ['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG']]

# Let's rename the columns and set index name as Home Team to do further analysis
df2.columns = ['Date', 'HomeTeam', 'AwayTeam', 'Winner', 'HomeTeamGoals', 'AwayTeamGoals']
#df2 = df2.set_index('HomeTeam')
df2.head()

,Date,HomeTeam,AwayTeam,Winner,HomeTeamGoals,AwayTeamGoals
0,11/08/17,Arsenal,Leicester,H,4,3
1,12/08/17,Brighton,Man City,A,0,2
2,12/08/17,Chelsea,Burnley,A,2,3
3,12/08/17,Crystal Palace,Huddersfield,A,0,3
4,12/08/17,Everton,Stoke,H,1,0


In [31]:
# ------------------- Home game dataset ---------------------
mancity_home = df2.loc[df2['HomeTeam'] == 'Man City'].reset_index()

mancity_home['Opponent']  = mancity_home['AwayTeam']
mancity_home['HomeMatch'] = True
mancity_home['GoalsScored']   = mancity_home['HomeTeamGoals']
mancity_home['GoalsConceded'] = mancity_home['AwayTeamGoals']
mancity_home['Result'] = np.where(mancity_home['Winner'] == 'D', 'draw', 
                                  np.where(mancity_home['Winner'] == 'H', 'win', 'loss'))

mancity_home.drop(['index','HomeTeam', 'AwayTeam', 'HomeTeamGoals', 'AwayTeamGoals','Winner'], axis=1, inplace=True)
mancity_home.set_index('Date', inplace=True)


# ------------- Away Game dataset --------------------------
mancity_away = df2.loc[df2['AwayTeam'] == 'Man City'].reset_index()

mancity_away['Opponent'] = mancity_away['HomeTeam']
mancity_away['HomeMatch'] = False
mancity_away['GoalsScored'] = mancity_away['AwayTeamGoals']
mancity_away['GoalsConceded'] = mancity_away['HomeTeamGoals']

mancity_away['Result'] = np.where(mancity_away['Winner'] == 'D', 'draw', 
                                  np.where(mancity_away['Winner'] == 'A', 'win', 'loss'))

mancity_away.drop(['index', 'HomeTeam', 'AwayTeam', 'HomeTeamGoals', 'AwayTeamGoals','Winner'], axis=1, inplace=True)
mancity_away.set_index('Date', inplace=True)


mancity = pd.concat([mancity_home, mancity_away]) 
mancity.head()

,Opponent,HomeMatch,GoalsScored,GoalsConceded,Result
Date,,,,,
21/08/17,Everton,True,1,1,draw
09/09/17,Liverpool,True,5,0,win
23/09/17,Crystal Palace,True,5,0,win
14/10/17,Stoke,True,7,2,win
21/10/17,Burnley,True,3,0,win


In [13]:
# We can do some intersting stats 
goaldiff = mancity['GoalsScored'].sum() - mancity['GoalsConceded'].sum()
matchesplayed  = mancity['Result'].count()
wins = (mancity['Result'] == 'win').sum()
losses = (mancity['Result'] == 'loss').sum()
draws = (mancity['Result'] == 'draw').sum()

print(" ** Man City Stats for 2017 ** \n Matches Played:{0} \t Matches Won:{1} \t Draws:{2} \t Losses:{3}"
      .format(matchesplayed, wins, draws, losses))

 ** Man City Stats for 2017 ** 
 Matches Played:23 	 Matches Won:20 	 Draws:2 	 Losses:1


### Filtering

Let's look at how we filtered out some data from the raw dataset 'epl'. 
We create a binary series that can be used as filter

In [9]:
# Create a binary Series as filter
flt = df2['HomeTeam'] == 'Man City'
flt.head()

0    False
1    False
2    False
3    False
4    False
Name: HomeTeam, dtype: bool

We can join more than one filter condition using logical operations - and (&) , or(|)

In [29]:
# Using Either conditions to get all matches played by Man City
mancity_flt = (df2['HomeTeam'] == 'Man City') | (df2['AwayTeam'] == 'Man City')

df2[ mancity_flt ]

,Date,HomeTeam,AwayTeam,Winner,HomeTeamGoals,AwayTeamGoals
1,12/08/17,Brighton,Man City,A,0,2
19,21/08/17,Man City,Everton,D,1,1
20,26/08/17,Bournemouth,Man City,A,1,2
34,09/09/17,Man City,Liverpool,H,5,0
46,16/09/17,Watford,Man City,A,0,6
53,23/09/17,Man City,Crystal Palace,H,5,0
61,30/09/17,Chelsea,Man City,A,0,1
73,14/10/17,Man City,Stoke,H,7,2
83,21/10/17,Man City,Burnley,H,3,0
96,28/10/17,West Brom,Man City,A,2,3


In [28]:
# using Both conditions to get all matches where Man City won
winflt1 = (df2['HomeTeam'] == 'Man City') & (df2['Winner'] == 'H')
winflt2 = (df2['AwayTeam'] == 'Man City') & (df2['Winner'] == 'A')

df2[ winflt1 | winflt2 ]

,Date,HomeTeam,AwayTeam,Winner,HomeTeamGoals,AwayTeamGoals
1,12/08/17,Brighton,Man City,A,0,2
20,26/08/17,Bournemouth,Man City,A,1,2
34,09/09/17,Man City,Liverpool,H,5,0
46,16/09/17,Watford,Man City,A,0,6
53,23/09/17,Man City,Crystal Palace,H,5,0
61,30/09/17,Chelsea,Man City,A,0,1
73,14/10/17,Man City,Stoke,H,7,2
83,21/10/17,Man City,Burnley,H,3,0
96,28/10/17,West Brom,Man City,A,2,3
108,05/11/17,Man City,Arsenal,H,3,1


In [36]:
# Now let's use filters to retrieve all mancity matches where both teams scored
goals_flt = (mancity['GoalsScored'] > 0 ) & (mancity['GoalsConceded'] > 0)
mancity[goals_flt]

,Opponent,HomeMatch,GoalsScored,GoalsConceded,Result
Date,,,,,
21/08/17,Everton,True,1,1,draw
14/10/17,Stoke,True,7,2,win
05/11/17,Arsenal,True,3,1,win
29/11/17,Southampton,True,2,1,win
03/12/17,West Ham,True,2,1,win
16/12/17,Tottenham,True,4,1,win
02/01/18,Watford,True,3,1,win
26/08/17,Bournemouth,False,2,1,win
28/10/17,West Brom,False,3,2,win


,GoalsConceded
Opponent,
Everton,1
Liverpool,0
Crystal Palace,0
Stoke,2
Burnley,0
Arsenal,1
Southampton,1
West Ham,1
Tottenham,1
